# Linear classification

A linear classifier achieves its classification decision $\hat{y_i}$ based on the value of a linear combination of the input features of a given sample $x_i$.
$$
\hat{y_i} = f(w \cdot x_i)
$$
Where $w \cdot x_i ::= w^Tx_i$ is the dot product bewteen $w$ and $x_i$.

Linear classifers learners looks for a weights vector $w$ that minimizes a given loss function: $L(y_i, \hat{y_i})$ and some penalties on the weights vector.

## Fisher's linear discriminant with equal class covariance

This geometric method does not make probabilistic assumption, it only relies on distances. It look for the **linear projection** (rotation) $\mathbf{w}$ that maximizes the between / within variance ratio: noted $F(w)$. It should be considered as a pedagogical experimental method. However with few assumptions it will provide the same results than Linear discriminant analysis (LDA) explained below.

Suppose two classes ($C_0, C_1$) of observations have means $\mu_0$, $\mu_1$ and the same the 
total within-class scatter ``covariance'' matrix $S_W$ given by:
\begin{align}
S_W &= \sum_{i\in C_0} (x_i - \mu_0)(x_i - \mu_0)^T + \sum_{j\in C_1} (x_j - \mu_1)(x_j -\mu_1)^T\\
    &= X_c^T X_c
\end{align}

Where $X_c$ is the $(N \times P)$ matrix of data centered on their respective means:

$$
X_c = 
\begin{bmatrix}
X_0 -  \mu_0 \\ X_1 -  \mu_1 
\end{bmatrix}
$$

Where $X_0$ and $X_1$ are the $(N_0 \times P)$ and $(N_1 \times P)$ matrices of samples of classes $C_0$ and $C_1$.

Let $S_B$ being the scatter ``between-class'' covariance matrix and given by

$$
S_B = (\mu_1 - \mu_0 )(\mu_1 - \mu_0 )^T
$$


The linear combination of features $w^T x$ have means $w^T \mu_i$ for i=0,1 and variance $w^T 
X^T_c X_c w$. Fisher defined the separation between these two distributions to be the ratio of the 
variance between the classes to the variance within the classes:

\begin{align}
F_{\text{Fisher}}(w) &= \frac{\sigma_{\text{between}}^2}{\sigma_{\text{within}}^2}\\
                     &= \frac{(w^T \mu_1 - w^T \mu_0)^2}{w^T  X^T_c X_c w}\\
                     &= \frac{(w^T (\mu_1 - \mu_0))^2}{w^T  X^T_c X_c w}\\ 
                     &= \frac{w^T (\mu_1 - \mu_0) (\mu_1 - \mu_0)^T w}{w^T X^T_c X_c w}\\
                     &= \frac{w^T S_B w}{w^T S_W w}
\end{align}

### The Fisher most discriminant projection

In the two classes case, the maximum separation occurs by a projection on the $(\mu_1 - \mu_0)$ using the Mahalanobis 
metric $S_B^{-1}$:

$$
    w \propto S_W^{-1}(\mu_1 - \mu_0)
$$

#### Demonstration

Differentiating $F_{Fisher}(w)$ with respect to $w$

\begin{align*}
\nabla_{w}F_{Fisher}(w) &= 0\\
\nabla_{w}(\frac{w^T S_B w}{w^T S_W w}) &= 0\\
(w^T S_W w)(2 S_B w) - (w^T S_B w)(2 S_W w) &= 0\\
(w^T S_W w)(S_B w) &= (w^T S_B w)(S_W w)\\
S_B w &= \frac{w^T S_B w}{w^T S_W w}(S_W w)\\
S_B w &= \lambda (S_W w)\\
S_W^{-1}{S_B} w &= \lambda  w\\
\end{align*}

Since we do not care about the magnitude of $w$, only its direction, we replaced the scalar factor $(w^T S_B w) / (w^T S_W w)$ by $\lambda$. 

In the multiple $K$-classes case, the solutions $w$ are determined by the eigenvectors of $S_W^{-1}{S_B}$ that correspond to the ($K-1$) largest eigenvalues.

However, in the two classes case (where $S_B = (\mu_1 - \mu_0 )(\mu_1 - \mu_0 )^T$) it is easy to 
show that $w = S_W^{-1}(\mu_1 - \mu_0)$ is the unique eigenvector of $S_W^{-1}{S_B}$:

\begin{align*}
S_W^{-1}(\mu_1 - \mu_0 )(\mu_1 - \mu_0 )^T w &= \lambda  w\\
S_W^{-1}(\mu_1 - \mu_0 )(\mu_1 - \mu_0 )^T S_W^{-1}(\mu_1 - \mu_0) &= \lambda  S_W^{-1}(\mu_1 
- \mu_0)\\
\end{align*}

Where here $\lambda = (\mu_1 - \mu_0 )^T S_W^{-1}(\mu_1 - \mu_0)$. Which leads to the result:
$$
    w \propto S_W^{-1}(\mu_1 - \mu_0)
$$

### The separating hyperplane

The separating hyperplane is a plan $P-1$-dimensional hyper surface orthogonal to the projection vector: $w$. To define the origin of the plane along $w$. This origin is the classification threshold to decide whether a point should be classified $C_0$ or $C_1$. The threshold can be chosen as the hyperplane between projections of the two means:
$$
T = w \cdot \frac{1}{2}(\mu_1 - \mu_0)
$$

![The Fisher most discriminant projection](images/fisher_linear_disc.png)

### Exercise

Write an class ``FisherLinearDiscriminant`` that implements the Fisher's linear discriminant analysis. This class must be compliant with the scikit-learn API by providing two methods: 
- ``fit(X, y)`` which fits the model and returns the object itself;
- ``predict(X)`` which returns a vector of the predicted values.
Apply the objet on the dataset presented for the LDA.

## Linear discriminant analysis (LDA)

Linear discriminant analysis (LDA) is a probabilistic generalization of Fisher's linear discriminant. It uses Bayes' rule to fix the threshold based on prior probabilities of classes. 

1. First compute the The class-**conditional distributions** of $x$ given class $C_k$: $p(x|C_k) = \mathcal{N}(x|\mu_k, S_W)$. Where $\mathcal{N}(x|\mu_k, S_W)$ is the multivariate Gaussian distribution defined over a P-dimensional vector $x$ of continuous variables, which is given by
$$
\mathcal{N}(x|\mu_k, S_W) = \frac{1}{(2\pi)^{P/2}|S_W|^{1/2}}\exp\{-\frac{1}{2} (x - \mu_k)^T S_W^{-1}(x - \mu_k)\}
$$

2. Estimate the **prior probabilities** of class $k$, $p(C_k) = N_k/N$.

3. Compute **posterior probabilities** (ie. the probability of a each class given a sample) combining conditional with priors using Bayes' rule:
$$
p(C_k|x) = \frac{p(C_k) p(x|C_k)}{p(x)}
$$
Where $p(x)$ is the marginal distribution obtained by suming of classes:
As usual, the denominator
in Bayes’ theorem can be found in terms of the quantities appearing in the
numerator, because
$$
p(x) = \sum_k p(x|C_k)p(C_k)
$$

4. Classify $x$ using the Maximum-a-Posteriori probability: $C_k= \arg \max_{C_k} p(C_k|x)$

LDA is a **generative model** since the class-conditional distributions cal be used to generate samples of each classes.

LDA is useful to deal with imbalanced group sizes (eg.: $N_1 \gg N_0$) since priors probabilities can be used to explicitly re-balance the classification by setting $p(C_0) = p(C_1) = 1/2$ or whatever seems relevant.

LDA can be generalised to the mulicasses case with $K>2$.

With  $N_1 = N_0$, LDA lead to the same solution than Fisher's linear discriminant.

### Exercise

How many parameters are required to estimate to perform a LDA ?

In [1]:
%matplotlib inline

import numpy as np
from sklearn.lda import LDA

# Dataset
n_samples, n_features = 100, 2
mean0, mean1 = np.array([0, 0]), np.array([0, 2])
Cov = np.array([[1, .8],[.8, 1]])
np.random.seed(42)
X0 = np.random.multivariate_normal(mean0, Cov, n_samples)
X1 = np.random.multivariate_normal(mean1, Cov, n_samples)
X = np.vstack([X0, X1])
y = np.array([0] * X0.shape[0] + [1] * X1.shape[0])

# LDA with scikit-learn
lda = LDA()
proj = lda.fit(X, y).transform(X)
y_pred_lda = lda.predict(X)

errors =  y_pred_lda != y
print("Nb errors=%i, error rate=%.2f" % (errors.sum(), errors.sum() / len(y_pred_lda)))

Nb errors=10, error rate=0.05


/usr/local/anaconda3/lib/python3.5/site-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)


## Logistic regression

Logistic regression is called a generalized linear models. ie.: it is a linear model with a link function that maps the output of linear multiple regression to a the posterior probability of each class $p(C_k|x) \in [0, 1]$ using the logistic sigmoid function:

$$
p(C_k|w, x_i) = \frac{1}{1 + \exp(-w \cdot x_i)}
$$

![logistic sigmoid function](images/logistic.png)

Logistic regression seeks to minimizes the likelihood as **Loss function**:

\begin{array}{lll}
\text{min}~L(w) &= \Pi_i^N p(C_k|w, x_i)\\
& \text{Or log-likelihood:}\\
\text{min}~\log L(w) &= \sum_i^N \log p(C_k|w, x_i)\\
\end{array}


\begin{align}

\end{align}

In the two-class case the algorithms simplify considerably by coding the two-classes ($C_0$ and $C_1$) via a 0/1 response $y_i$. Indeed, since $p(C_0|w, x_i) = 1 - p(C_1|w, x_i)$, the log-likelihood can be re-witten:

\begin{align}
\log L(w) &=  \sum_i^N\{y_i \log p(C_1|w, x_i) + (1 - y_i) \log(1 - p(C_1|w, x_i))\}\\
\log L(w) &= \sum_i^N\{y_i~w \cdot x_i - \log(1 + \exp^{w \cdot x_i})\}\\
\end{align}

Logistic regression is a **discriminative model** since it focuses only on the posterior probability of each class $p(C_k|x)$. It only requires to estimate the $P$ weight of the $w$ vector. Thus it should be favoured over LDA with many input features. In small dimension and balanced situations it would provide similar predictions than LDA.

However imbalanced group sizes cannot be explicitly controlled. It can be managed using a reweighting of the input samples.

In [2]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression(C=1e8)
# This class implements regularized logistic regression. C is the Inverse of regularization strength.
# Large value => no regularization.

logreg.fit(X, y)
y_pred_logreg = logreg.predict(X)

errors =  y_pred_logreg != y
print("Nb errors=%i, error rate=%.2f" % (errors.sum(), errors.sum() / len(y_pred_logreg)))
print(logreg.coef_)

Nb errors=10, error rate=0.05
[[-5.15162649  5.57299286]]


### Exercise

Explore the ``Logistic Regression`` parameters and proposes a solution in cases of highly imbalanced training dataset $N_1 \gg N_0$  when we know that in reality both classes have the same probability $p(C_1) = p(C_0)$.

## Ridge Fisher's linear classification (L2-regularization)

When the matrix $S_W$ is not full rank or $P \gg N$, the The Fisher most discriminant projection estimate of the is not unique. This can be solved using a biased version of $S_W$:
$$
S_W^{Ridge} = S_W + \lambda I
$$

where $I$ is the $P \times P$ identity matrix. This leads to the regularized (ridge) estimator of the Fisher's linear discriminant analysis: 
$$
    w^{Ridge} \propto (S_W + \lambda I)^{-1}(\mu_1 - \mu_0)
$$

![The Ridge Fisher most discriminant projection](images/ridge_fisher_linear_disc.png)

Increasing $\lambda$ will:

- Shrinks the coeficients toward zero.
- The covariance will converge toward the diagnonal matrix, reducing the contribution of the pairwise covariances.

## Ridge logistic regression (L2-regularization)

The **objective function** to be minimized is now the combination of the logistic loss $\log L(w)$ with a penalty of the L2 norm of the weights vector. In the the two-class case, using the 0/1 coding we obtain:

\begin{align}
\min~F_{\text{Logistic Ridge}}(w) &= \text{Loss}(w)  + \lambda~\text{penalty}(w)\\
                 &= \log L(w) + \lambda~\text{Euclidian norm}(w)^2\\
                 &= \sum_i^N\{y_i w^T x_i - \log(1 + \exp^{w^Tx_i})\} + \lambda~w \cdot w
\end{align}


In [3]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression(C=1)
# This class implements regularized logistic regression. C is the Inverse of regularization strength.
# Large value => no regularization.

logreg.fit(X, y)
y_pred_logreg = logreg.predict(X)

errors =  y_pred_logreg != y
print("Nb errors=%i, error rate=%.2f" % (errors.sum(), errors.sum() / len(y_pred_logreg)))
print(logreg.coef_)

Nb errors=11, error rate=0.06
[[-2.44483173  2.89508254]]


## Lasso logistic regression (L1-regularization)

The **objective function** to be minimized is now the combination of the logistic loss $\log L(w)$ with a penalty of the L1 norm of the weights vector. In the the two-class case, using the 0/1 coding we obtain:

\begin{align}
\min F_{\text{Logistic Ridge}}(w) &= \log L(w) + \lambda~\text{L1 norm}(w)\\
                 &= \sum_i^N\{y_i~w \cdot x_i - \log(1 + \exp^{w \cdot x_i})\} + \lambda~||w||_1
\end{align}

In [4]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression(penalty='l1', C=1)
# This class implements regularized logistic regression. C is the Inverse of regularization strength.
# Large value => no regularization.

logreg.fit(X, y)
y_pred_logreg = logreg.predict(X)

errors =  y_pred_logreg != y
print("Nb errors=%i, error rate=%.2f" % (errors.sum(), errors.sum() / len(y_pred_logreg)))
print(logreg.coef_)

Nb errors=11, error rate=0.06
[[-3.46482319  3.92907446]]


## Ridge linear Support Vector Machine (L2-regularization)

Support Vector Machine seek for separating hyperplane with maximum margin to enforce robustness against noise.

Here we present the non separable case of Maximum Margin Classifiers with $\pm 1$ coding (ie.: $y_i \ \{-1, +1\}$).

![Linear lar margin classifiers](images/svm.png)

Linear SVM for classification (also called SVM-C or SVC) minimizes:

\begin{array}{lll}
\text{min}   & F_{\text{Linear SVM}}(w) &= \text{penalty}(w) +  C~\text{Loss}(w)\\
             & & = \lambda~||w||_2 + C~\sum_i^N\xi_i\\
\text{with}  & \forall i & y_i (w \cdot x_i) \geq 1 - \xi_i
\end{array}

Here we introduced the slack variables: $\xi_i$, with $\xi_i = 0$ for points that are on or inside the correct margin boundary and $\xi_i = |y_i - (w \ cdot  \cdot x_i)|$ for other points. Thus:

1. If $y_i (w \cdot x_i) \geq 1$ then the point lies outside the margin but on the correct side of the decision boundary. In this case $\xi_i=0$. The constraint is thus not active for this point. It does not contribute to the prediction.

2. If $1 > y_i (w \cdot x_i) \geq 0$ then the point lies inside the margin and on the correct side of the decision boundary. In this case $0<\xi_i \leq 1$. The constraint is active for this point. It does contribute to the prediction as a support vector.

3. If $0 <  y_i (w \cdot x_i)$) then the point is on the wrong side of the decision boundary (missclassification). In this case $0<\xi_i > 1$. The constraint is active for this point. It does contribute to the prediction as a support vector.

This loss is called the hinge loss, defined as:

$$
\max(0, 1 - y_i~ (w \cdot x_i))
$$

So linear SVM is closed to Ridge logistic regression, using the hing loss instead of the logistic loss. Both will provide very similar predictions.

In [5]:
from sklearn import svm

svmlin = svm.LinearSVC()
# Remark: by default LinearSVC uses squared_hinge as loss
svmlin.fit(X, y)
y_pred_svmlin = svmlin.predict(X)

errors =  y_pred_svmlin != y
print("Nb errors=%i, error rate=%.2f" % (errors.sum(), errors.sum() / len(y_pred_svmlin)))
print(svmlin.coef_)

Nb errors=10, error rate=0.05
[[-1.2958853   1.44125835]]


## Lasso linear Support Vector Machine (L1-regularization)

Linear SVM for classification (also called SVM-C or SVC) with l1-regularization

\begin{array}{lll}
\text{min}   & F_{\text{Lasso linear SVM}}(w) &= \lambda~||w||_1 + C~\sum_i^N\xi_i\\
\text{with}  & \forall i & y_i (w \cdot x_i) \geq 1 - \xi_i
\end{array}


In [6]:
from sklearn import svm

svmlinl1 = svm.LinearSVC(penalty='l1', dual=False, C=.02)
# Remark: by default LinearSVC uses squared_hinge as loss

svmlinl1.fit(X, y)
y_pred_svmlinl1 = svmlinl1.predict(X)

errors =  y_pred_svmlinl1 != y
print("Nb errors=%i, error rate=%.2f" % (errors.sum(), errors.sum() / len(y_pred_svmlinl1)))
print(svmlinl1.coef_)

Nb errors=11, error rate=0.06
[[-0.3340551   0.50440453]]
